In [1]:
#!pip install selenium

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import csv
import re
import pickle
import os
import random
import pandas as pd
import numpy as np

In [3]:
# Функция авторизации
def auth(driver):
    # Авторизация по cookies
    if os.path.exists(FILES_PATH + COOKIES_PATH):
        driver.get('https://linkedin.com')
        #Загрузка куки
        for cookie in pickle.load(open(FILES_PATH + COOKIES_PATH, 'rb')):
            driver.add_cookie(cookie)
        time.sleep(random.uniform(.5, 1))
        driver.refresh()
        time.sleep(random.uniform(.5, 2))

    # Вход по паролю и сохранение cookies
    else:
        driver.get('https://linkedin.com/uas/login')
        #Авторизация
        time.sleep(random.uniform(2, 4))
        username = driver.find_element(By.ID, "username")
        username.send_keys(str(input('Введите логин: ')))
        pword = driver.find_element(By.ID, "password")
        pword.send_keys(str(input('Введите пароль: ')))
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        time.sleep(30) #время на ввод кода подтверждения если понадобится
        pickle.dump(driver.get_cookies(), open(FILES_PATH + COOKIES_PATH, 'wb'))

In [4]:
# Функция для получения URL страниц пользователей
def profile_urls(driver, search_url):
    # Загрузка страницы с результатами поиска
    driver.get(search_url)
    time.sleep(random.uniform(.5, 3))

    # Необходимые переменные
    profile_urls = []

    for i in range(NUM_PAGES_TO_PARSE):
        # Текущая страница
        print('Page', i+1)
        # Задержка для полной загрузки страницы
        time.sleep(random.uniform(4, 7))

        search_result_links = driver.find_elements(By.CSS_SELECTOR, "span.entity-result__title-text a.app-aware-link")

        for link in search_result_links:
            href = link.get_attribute("href")
            if 'linkedin.com/in' in href:
                profile_urls.append(href)

        last_height = driver.execute_script("return document.body.scrollHeight")
        NUM_SCROLLS = 20

        for _ in range(NUM_SCROLLS):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(random.uniform(1.5, 3))
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        time.sleep(random.uniform(4, 7))

        print('Sucсess!')
        print('-' * 50)

        next_button = driver.find_element(By.CSS_SELECTOR, 'button.artdeco-pagination__button--next')
        next_button.click()


    profile_urls = pd.Series(profile_urls)
    profile_urls_cut = profile_urls.str.split('?', n=1).str[0]
    print('Done!')

    # Сохранение файла с URL
    profile_urls_cut.to_csv(URL_FILE_NAME)
    print(f'URLs saved to file: {URL_FILE_NAME}')

In [16]:
def get_and_save_profile_info(driver, profile_url):
    info_source = []
    
    # This will open the link
    driver.get(profile_url)
    
    # Random sleeping time to load all data
    time.sleep(random.uniform(5, 7))

    # Extracting data from page with BeautifulSoup
    src = driver.page_source

    # Now using beautiful soup    
    soup = BeautifulSoup(src, 'lxml')

    # Extracting the HTML of the complete introduction box
    # that contains the name, status, and the location
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})
    
    # Extracting the Name
    # In case of an error, try changing the tags used here.
    name_loc = intro.find("h1")
    # strip() is used to remove any extra blank spaces
    name = name_loc.get_text().strip()
    info_source.append(name)

    # This gives us the HTML of the tag in which user status is present
    status_loc = intro.find("div", {'class': 'text-body-medium'})
    # Extracting user status
    status = status_loc.get_text().strip()
    info_source.append(status)
    
    # Extracting the Company name
    work_space = soup.find('ul', {'class': 'pv-text-details__right-panel'})
    # This gives us the HTML of the tag in which the Company Name is present
    works_at_loc = work_space.find('span', {'class': 'pv-text-details__right-panel-item-text'})
    # Extracting the Company Name
    works_at = works_at_loc.get_text().strip()
    info_source.append(works_at)

    # Добавил случайный скроллинг страницы для имитации человека
    last_height = driver.execute_script("return document.body.scrollHeight")
    NUM_SCROLLS = random.randint(4, 10)

    for _ in range(NUM_SCROLLS):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(1.5, 3))
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(random.uniform(2, 5))
    
    # Add user profile URL
    info_source.append(profile_url)
    
    # Сохранение в файл
    with open(INFO_FILE_NAME, 'a', newline='', encoding='utf-8') as file:
        writer_obj = csv.writer(file)
        writer_obj.writerow(info_source)

    # Print collected data
#     print("Name -->",  name,
#           "\nStatus -->", status,
#           "\nWorks At -->", works_at,
#           "\nProfile URL -->", cur_profile_url)   

In [6]:
def get_and_save_users_posts(driver, url):
    driver.get(url + '/recent-activity/all/')
    time.sleep(random.uniform(4, 7))
    SCROLL_PAUSE_TIME = random.uniform(1, 3)
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # We can adjust this number to get more posts
    NUM_SCROLLS = random.randint(7, 10)
    for i in range(NUM_SCROLLS):
    
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(random.uniform(3,6))

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

        last_height = new_height

    src = driver.page_source
    
    # Установил lxml
    soup = BeautifulSoup(src, 'lxml')
    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')

    for post_src in posts:
        post_source = []
        post_source.append(url)

        # Текст статьи репоста
        post_text_div = post_src.find('a', {'class': 'tap-target update-components-mini-update-v2__link-to-details-page text-body-medium ember-view'})
        if post_text_div is not None:
            post_text = post_text_div.find('span', {'dir': 'ltr'})
        else:
            post_text = None

        if post_text is not None:
            post_text = post_text.get_text().strip()

        if post_text is None:
            post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})            
            if post_text_div is not None:
                post_text = post_text_div.find('span', {'dir': 'ltr'})
            else:
                post_text = None

            # If post text is found
            if post_text is not None:
                post_text = post_text.get_text().strip()
            else:
                post_text = 'No text'

        post_source.append(post_text)

        # Подсчет лайков
        likes_cnt = post_src.find('span', {'class': 'social-details-social-counts__reactions-count'})

        # If number of reactions is written as text
        # It has different class name
        if likes_cnt is None:
            likes_cnt = post_src.find('span', {'class': 'social-details-social-counts__social-proof-text'})


        if likes_cnt is not None:
            likes_cnt = likes_cnt.get_text().strip()
        else:
            likes_cnt = 0

        post_source.append(likes_cnt)

        # Подсчет репостов
        reposts_cnt = post_src.find('li', {'class': 'social-details-social-counts__item social-details-social-counts__item--with-social-proof'})
        if reposts_cnt is not None:
            reposts_cnt = reposts_cnt.find('span', {'aria-hidden': 'true'})
        if reposts_cnt is not None:
            reposts_cnt = reposts_cnt.get_text().strip()
        else:
            reposts_cnt = 0 
        post_source.append(reposts_cnt)

        # Подсчет комментариев
        comment_cnt = post_src.find('li', {'class': 'social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof'})
        if comment_cnt is not None:
            comment_cnt = comment_cnt.find('span', {'aria-hidden': 'true'})
        if comment_cnt is not None:
            comment_cnt = comment_cnt.get_text().strip()
        else:
            comment_cnt = 0 

        post_source.append(comment_cnt)          
        
        # Сохранение в файл
        with open(POSTS_FILE_NAME, 'a', newline='', encoding='utf-8') as file:
            writer_obj = csv.writer(file)
            writer_obj.writerow(post_source)
            
        time.sleep(random.uniform(2, 5))

In [7]:
# Количество страниц результатов поиска для парсинга
NUM_PAGES_TO_PARSE = 17

# Название файла cookies
COOKIES_PATH = '\lincookies'

# Папка для хранения файлов
FILES_PATH = 'C:\Recruiters'

# Путь к файлу с URL
URL_FILE_NAME = FILES_PATH + r"\urls.csv"

# Путь к файлу с инфо пользователей
INFO_FILE_NAME = FILES_PATH + r"\user-info.csv"

# Путь к файлу с постами
POSTS_FILE_NAME = FILES_PATH + r"\posts.csv"

# Суффикс ссылки на посты пользователя
POSTS_URL_SUFFIX = 'recent-activity/all/'

# Ввод поисковой строки
SEARCH_URL = str(input('Введите строку поиска: '))

Введите строку поиска: https://www.linkedin.com/search/results/people/?currentCompany=%5B%2235639643%22%2C%228979%22%2C%22531312%22%2C%226132%22%2C%2225880%22%2C%2210718%22%2C%2277009034%22%2C%22970369%22%2C%22868912%22%2C%22579461%22%2C%2210039597%22%2C%2210559285%22%2C%2210876016%22%2C%22204630%22%2C%2279365269%22%2C%2279816292%22%2C%2280856181%22%5D&geoUrn=%5B%22101705918%22%2C%22104994045%22%2C%22106686604%22%2C%22106049128%22%2C%22107734735%22%5D&keywords=lead%20data%20scientist&origin=FACETED_SEARCH&searchId=25003ca8-2d98-45ab-b23f-280d8c230c99&sid=M1r


In [8]:
# Запуск драйвера для скрэпинга

# Проверка наличия папки для файлов
if not os.path.exists(FILES_PATH):
    os.mkdir(FILES_PATH)

caps = DesiredCapabilities().CHROME
caps['pageLoadStrategy'] = 'eager'
driver = webdriver.Chrome()

# Установлен широкоформатный размер экрана, чтобы нужные элементы помещались на экране
driver.set_window_size(1920, 1080)

# Вызов функции авторизации
auth(driver)
time.sleep(random.uniform(5, 12))

In [9]:
# Парсинг URL адресов страниц пользователей и сохранение их в файл
profile_urls(driver, SEARCH_URL)
time.sleep(random.uniform(5, 12))

Page 1
Sucсess!
--------------------------------------------------
Page 2
Sucсess!
--------------------------------------------------
Page 3
Sucсess!
--------------------------------------------------
Page 4
Sucсess!
--------------------------------------------------
Page 5
Sucсess!
--------------------------------------------------
Page 6
Sucсess!
--------------------------------------------------
Page 7
Sucсess!
--------------------------------------------------
Page 8
Sucсess!
--------------------------------------------------
Page 9
Sucсess!
--------------------------------------------------
Page 10
Sucсess!
--------------------------------------------------
Page 11
Sucсess!
--------------------------------------------------
Page 12
Sucсess!
--------------------------------------------------
Page 13
Sucсess!
--------------------------------------------------
Page 14
Sucсess!
--------------------------------------------------
Page 15
Sucсess!
----------------------------------------

In [18]:
# Загрузка URL's из файла
profile_urls = pd.read_csv(URL_FILE_NAME, index_col=0, header=0, names=['user_url'])

#Создаем файл для записи постов
header = ['name', 'status', 'company', 'profile_url']
with open(INFO_FILE_NAME, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(header)

# Получение информации из профилей пользователей
for url in profile_urls['user_url']:
    get_and_save_profile_info(driver, url)
    time.sleep(random.uniform(4, 7))

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
profile_urls = pd.read_csv(URL_FILE_NAME, index_col=0, header=0, names=['user_url'])

#Создаем файл для записи постов
header = ['url', 'text', 'likes_cnt', 'reposts_cnt', 'comments_cnt']
with open(POSTS_FILE_NAME, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(header)

for url in profile_urls['user_url']:
    get_and_save_users_posts(driver, url)

In [ ]:
# Выключаем драйвер скрэппинга
driver.quit()